## 参考資料

- SparkのpackagesをSparkSession作成時に指定する方法: https://qiita.com/junkor-1011/items/b12ec62f2615d068c1a5
- Hudiクイックスタート: https://hudi.apache.org/docs/0.7.0-quick-start-guide.html#pyspark-example
- Hiveオプションについてのもの: https://dev.classmethod.jp/articles/apache-hudi-on-aws-sync-glue-data-catalog/

In [1]:
# 必要なSparkモジュールの読み込み
from pyspark.sql import SparkSession

In [2]:
# SparkSessionの作成
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hudi:hudi-spark-bundle_2.12:0.7.0,org.apache.spark:spark-avro_2.12:3.0.1") \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .getOrCreate()
sc = spark.sparkContext

In [3]:
# テーブル設定
tableName = "hudi_trips_cow"
basePath = "file:///tmp/hudi_trips_cow"
# サンプルデータ生成用ライブラリの読み込み
dataGen = sc._jvm.org.apache.hudi.QuickstartUtils.DataGenerator()

In [4]:
# データインサート
# インサート用のデータを作成
inserts = sc._jvm.org.apache.hudi.QuickstartUtils.convertToStringList(dataGen.generateInserts(10))
df = spark.read.json(spark.sparkContext.parallelize(inserts, 2))

# Hudiのオプション
hudi_options = {
    'hoodie.table.name': tableName,
    'hoodie.datasource.write.recordkey.field': 'uuid',
    'hoodie.datasource.write.partitionpath.field': 'partitionpath',
    'hoodie.datasource.write.table.name': tableName,
    'hoodie.datasource.write.operation': 'upsert', # upsertを指定しているため、後述の更新処理の際もこのオプションをそのまま利用できる
    'hoodie.datasource.write.precombine.field': 'ts',
    'hoodie.upsert.shuffle.parallelism': 2, 
    'hoodie.insert.shuffle.parallelism': 2,
    # 以下Hiveオプション(なお、現在Hiveを起動できないので使用できない)
    # 'hoodie.datasource.hive_sync.enable': 'true',
    # 'hoodie.datasource.hive_sync.database': 'default',
    # 'hoodie.datasource.hive_sync.table': tableName,
    # 'hoodie.datasource.hive_sync.partition_fields': 'contient,country,city',
    # 'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor',
    # 'hoodie.datasource.hive_sync.use_jdbc': 'false',
}

# データの書き込み
# 「mode」にoverwriteが指定されている場合、新規にテーブルを作成する。(すでにテーブル等が存在する場合は、Dropされる)
df.write.format("hudi"). \
  options(**hudi_options). \
  mode("overwrite"). \
  save(basePath)

In [5]:
# テーブル確認
%ls /tmp/hudi_trips_cow/

americas/  asia/


In [6]:
# データクエリ
# DataFrameを読み込む
tripsSnapshotDF = spark. \
  read. \
  format("hudi"). \
  load(basePath + "/*/*/*/*")
# load(basePath) use "/partitionKey=partitionValue" folder structure for Spark auto partition discovery

# DataFrameにテーブル名をつける(SQLのViewのようなもの)
tripsSnapshotDF.createOrReplaceTempView("hudi_trips_snapshot")

# クエリの実行
spark.sql("SELECT fare, begin_lon, begin_lat, ts FROM hudi_trips_snapshot WHERE fare > 20.0").show()
spark.sql("SELECT _hoodie_commit_time, _hoodie_record_key, _hoodie_partition_path, rider, driver, fare FROM hudi_trips_snapshot").show()

+------------------+-------------------+-------------------+-------------+
|              fare|          begin_lon|          begin_lat|           ts|
+------------------+-------------------+-------------------+-------------+
| 27.79478688582596| 0.6273212202489661|0.11488393157088261|1613220491019|
| 93.56018115236618|0.14285051259466197|0.21624150367601136|1612848023511|
| 33.92216483948643| 0.9694586417848392| 0.1856488085068272|1613195135367|
| 64.27696295884016| 0.4923479652912024| 0.5731835407930634|1613120973856|
|  43.4923811219014| 0.8779402295427752| 0.6100070562136587|1613296123130|
|34.158284716382845|0.46157858450465483| 0.4726905879569653|1613398114469|
| 66.62084366450246|0.03844104444445928| 0.0750588760043035|1613043859793|
| 41.06290929046368| 0.8192868687714224|  0.651058505660742|1613111667302|
+------------------+-------------------+-------------------+-------------+

+-------------------+--------------------+----------------------+---------+----------+-------------

In [7]:
# データの更新
# 更新用のデータ作成
updates = sc._jvm.org.apache.hudi.QuickstartUtils.convertToStringList(dataGen.generateUpdates(10))
df = spark.read.json(spark.sparkContext.parallelize(updates, 2))
# データ更新
df.write.format("hudi"). \
  options(**hudi_options). \
  mode("append"). \
  save(basePath)

In [8]:
# 更新確認
from pyspark.sql.functions import col
spark.sql("SELECT uuid, fare, begin_lon, begin_lat, ts FROM hudi_trips_snapshot ORDER BY uuid").show()
updatedDF = spark. \
  read. \
  format("hudi"). \
  load(basePath + "/*/*/*/*")
updatedDF.orderBy(col("uuid")).select("uuid", "fare", "begin_lon", "begin_lat", "ts").show()

+--------------------+------------------+-------------------+-------------------+-------------+
|                uuid|              fare|          begin_lon|          begin_lat|           ts|
+--------------------+------------------+-------------------+-------------------+-------------+
|07f54b00-6e67-490...|  43.4923811219014| 0.8779402295427752| 0.6100070562136587|1613296123130|
|11271645-1998-458...| 66.62084366450246|0.03844104444445928| 0.0750588760043035|1613043859793|
|2e5c4ca5-3f65-499...|19.179139106643607| 0.7528268153249502| 0.8742041526408587|1613220393034|
|34fb050a-d5d3-465...|17.851135255091155| 0.5644092139040959|   0.40613510977307|1613212362950|
|4993421c-3b43-4fb...|34.158284716382845|0.46157858450465483| 0.4726905879569653|1613398114469|
|4ac0e8ea-b348-4c6...| 64.27696295884016| 0.4923479652912024| 0.5731835407930634|1613120973856|
|6d776c96-10fe-4ec...| 41.06290929046368| 0.8192868687714224|  0.651058505660742|1613111667302|
|7e1c06b4-4855-4b1...| 27.79478688582596